# Thompson Sampling 

In [20]:
import numpy as np 
import matplotlib.pyplot as plt 

In [21]:
class GaussianArm:
    def __init__(self, mean, std):
        self.mean = mean
        self.std = std 

    def sample(self):
        return np.random.normal(self.mean, self.std)         

In [26]:
arms = [GaussianArm(5, 1), GaussianArm(6, 1)]
prior_mean = 0
prior_std = 100
def posterior(prior_std, n, reward_sum):
    post_std = np.sqrt(1/((1/prior_std**2) + n))
    post_mean = (post_std**2)*reward_sum
    return np.random.normal(post_mean, post_std)

In [23]:
N_visits = 1000
num_arms = len(arms)
num_visits = [ 0 for i in range(num_arms)]
reward_sums = [ 0 for i in range(num_arms)]
scores = []
for i in range(N_visits):
    post_samples = []
    for j in range(num_arms):
        sample = posterior(prior_std, num_visits[j], reward_sums[j])
        post_samples.append(sample)
    arm_chosen = np.argmax(post_samples)
    reward = arms[arm_chosen].sample()
    reward_sums[arm_chosen] += reward 
    num_visits[arm_chosen] += 1
    scores.append(reward)

In [24]:
num_visits

[11, 989]